In [1]:
# Standard imports
import numpy as np
import pandas as pd
import ast 

# For scraping
import requests
from bs4 import BeautifulSoup

# For adding delays so that we don't spam requests
import time

import string

import re
import json


In [2]:

def get_genre(str_dic):
    genre_list=[]
    for i in str_dic:
        if i == '"genre":':
            j=str_dic.index(i)+1
            while str_dic[j] not in ['],','"actor":','"director":','"creator":','"datePublished":','}</script>]']:
                for punctuation_mark in string.punctuation:
                    str_dic[j]=str_dic[j].replace(punctuation_mark,'')
                if str_dic[j]!="":   
                    genre_list.append(str_dic[j])
                j+=1
            break  

    return genre_list

In [3]:
df_sixties=pd.read_csv('df_sixties.csv')
df_sixties

,tconst,primaryTitle,startYear
0,tt0039965,Viva Chihuahua,1961
1,tt0045136,Semmelweis,1961
2,tt0048162,Herr Puntila and His Servant Matti,1960
3,tt0049360,The Imposter,1960
4,tt0050441,Romeo e Giulietta,1964
...,...,...,...
9786,tt8024684,Face of a Stranger,1964
9787,tt8711822,Losos Laken Dhurafa'a,1968
9788,tt8735998,Miramar,1969
9789,tt9742424,Shchit i mech: Film vtoroy,1968


In [4]:
scrap_df = pd.DataFrame(columns=['tconst', 'genre', 'plot'])
i=0

for tconst in df_sixties['tconst']:


    imdb_to_scrap = {'tconst':"", 'genre':[], 'plot':""}
    
    response2=requests.get(f'https://www.imdb.com/title/{tconst}/plotsummary')
    soup2 = BeautifulSoup(response2.content,'html.parser')
        
    if soup2.title.text.strip() != '404 Error - IMDb':
        plot=soup2.find('ul', id="plot-synopsis-content").text.strip()


        if plot!='It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.':

            imdb_to_scrap = {'tconst':tconst, 'genre':[], 'plot':plot}

            # Send a get request and assign the response to a variable
            response1 = requests.get(f'https://www.imdb.com/title/{tconst}/')
            soup1 = BeautifulSoup(response1.content,'html.parser')


            str_dic=str(soup1.findAll('script', {'type':'application/ld+json'})).split()
            imdb_to_scrap['genre']=[get_genre(str_dic)]


            df = pd.DataFrame.from_dict(imdb_to_scrap)
            scrap_df = scrap_df.append(df, ignore_index=True)

            #follow progress: print tconst every 50 iteration
            i+=1
            if i%50==0:
                print(f'after 50 append {tconst}')


scrap_df


after 50 append tt0054358
after 50 append tt0055399
after 50 append tt0056283
after 50 append tt0057443
after 50 append tt0058450
after 50 append tt0059557
after 50 append tt0060545
after 50 append tt0061781
after 50 append tt0063032
after 50 append tt0064536
after 50 append tt0176877
after 50 append tt0314678


,tconst,genre,plot
0,tt0052721,"[Comedy, Fantasy]",Cuba Drobbyan (Jiri Sovak) is a poor cavalier ...
1,tt0052781,"[Drama, War]","In WWII Italy, a woman puts herself and her lo..."
2,tt0052784,"[Comedy, Crime, Horror]",Pablo Morales (Arturo de Cordova) is a quiet d...
3,tt0052997,"[Comedy, Crime, Thriller]",A manhole opens at night in an empty street an...
4,tt0053401,"[Drama, Musical, Comedy]","Fast-pasted, almost plot less musical-comedy s..."
...,...,...,...
611,tt1248983,[Romance],"A wealthy businessman from Ooty, Vedachalam (T..."
612,tt1729195,"[Crime, Drama]","When Louba, a crooked casino owner blackmails ..."
613,tt1729217,"[Crime, Drama, Thriller]","A young policeman, Dave Hollis, comes across a..."
614,tt2786412,[Documentary],Documentary about the Maxwell Street Market in...


In [5]:
scrap_df

,tconst,genre,plot
0,tt0052721,"[Comedy, Fantasy]",Cuba Drobbyan (Jiri Sovak) is a poor cavalier ...
1,tt0052781,"[Drama, War]","In WWII Italy, a woman puts herself and her lo..."
2,tt0052784,"[Comedy, Crime, Horror]",Pablo Morales (Arturo de Cordova) is a quiet d...
3,tt0052997,"[Comedy, Crime, Thriller]",A manhole opens at night in an empty street an...
4,tt0053401,"[Drama, Musical, Comedy]","Fast-pasted, almost plot less musical-comedy s..."
...,...,...,...
611,tt1248983,[Romance],"A wealthy businessman from Ooty, Vedachalam (T..."
612,tt1729195,"[Crime, Drama]","When Louba, a crooked casino owner blackmails ..."
613,tt1729217,"[Crime, Drama, Thriller]","A young policeman, Dave Hollis, comes across a..."
614,tt2786412,[Documentary],Documentary about the Maxwell Street Market in...


In [6]:
scrap_df.to_csv ('scraped_df_sixties.csv', index = False, header=True)